<a href="https://colab.research.google.com/github/SY-256/llms-from-scratch/blob/main/notebooks/ch05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ラベルなしデータでの事前学習
- LLMが訓練中に生成したテキストの品質を評価するために、訓練データと検証データセットでの損失（誤差）を計算する。
- 訓練関数を実装し、LLMの事前学習を行う。
- LLMを引き続き訓練するための重みを保存し、LLMに読み込む。
- OpenAIから事前学習済みの重みを読み込む。

## 5.1 生成テキストモデルを評価する

In [ ]:
# GPTモデルの設定
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # 語彙のサイズ
    "context_length": 256, # コンテキストの長さ
    "emb_dim": 768,         # 埋め込み次元数
    "n_heads": 12,          # Attentionヘッドの数
    "n_layers": 12,         # 層の数
    "drop_rate": 0.1,       # ドロップアウト率
    "qkv_bias": False       # クエリ、キー、バリューの計算にバイアスを使用するか
}

In [ ]:
import tiktoken
import torch
import torch.nn as nn

tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
# Layer Normalizationの実装と適用をモジュール化

class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False) # unbiased=True 有偏分散
        norm_x = (x - mean) / torch.sqrt(var + self.eps) # 標準化（ゼロ除算にならないようにepsを加える）
        return self.scale * norm_x + self.shift

# GELU活性化関数の実装
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))

# フォードフォワードニューラルネットワークモジュール
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]), # 1つ目の線形層で入力の次元から4倍にして出力
            GELU(), # GELU活性化関数
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]), # 2つ目の線形層で4倍にした入力の次元数を4分の1の元の次元数に戻して出力
        )

    def forward(self, x):
        return self.layers(x)

# 3章のMulti-head Attentionコンポーネント
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0) # 出力に次元数がヘッド数と割り切れない場合は警告出す

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # 出力次元数をヘッド数で分割

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.out_proj = nn.Linear(d_out, d_out) # Linear層を使ってヘッドの出力を組み合わせる
        self.dropout = nn.Dropout(dropout)
        # maskを作成 (context_lengthの形状に従う)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)

        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape # テンソルの形状は(batch, num_tokens, d_out)

        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        # num_heads次元を追加して行列えお暗黙的に分割
        # 最後の次元を展開し、形状を(batch, num_tokens, d_out) -> (batch, num_tokens, num_heads, head_dim)に変換
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # 形状を(batch, num_tokens, num_heads, head_dim) ->
        # (batch, num_heads, num_tokens, head_dim)に変換
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attn_scores = queries @ keys.transpose(2, 3) # 各ヘッドのドット積を計算

        mask_bool = self.mask.bool()[:num_tokens, :num_tokens] # マスクをトークン数で切り捨て

        attn_scores.masked_fill(mask_bool, -torch.inf) # Attentionスコアを埋めるためにマスクを使う

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2) # テンソルの形状は(batch, num_tokens, n_heads, head_dim)
        context_vec = context_vec.contiguous().view(
            b, num_tokens, self.d_out
        ) # self.d_out = self.num_heads * self.head_dimに基づいてヘッドを結合

        context_vec = self.out_proj(context_vec) # 線形射影を追加

        return context_vec

# GPTアーキテクチャのTransformerブロックコンポーネント
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"]
        )
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):

        shortcut = x # Attentionブロックのショートカット接続(入力のオリジナル)
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_shortcut(x)
        x = x + shortcut # 元の入力をショートカット接続として入力

        shortcut = x # フィードフォワードブロックのショートカット接続(前段の処理結果)
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        return x

# モデルアーキテクチャの実装
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        # Transformerブロック
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)

        # デバイス設定
        pos_embeds = self.pos_emb(
            torch.arange(seq_len, device=in_idx.device)
        )
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)

        return logits

In [ ]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()

コンテキストの長さ（context_length）を1024->256に短縮したことで、モデルの訓練に必要な計算量が少なくなり、標準的なラップトップコンピュータでも計算可能になる

In [ ]:
# テキストをトークンIDに変換するユーティリティ関数

# テキストを生成するGPTモデルの関数
def generate_text_simple(model, idx, max_new_tokens, context_size):
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:] # サポートされているコンテキストサイズを超える場合は現在のコンテキストを切り詰める
        with torch.no_grad():
            logits = model(idx_cond)

        logits = logits[:, -1, :]

        probas = torch.softmax(logits, dim=-1)
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)

        idx = torch.cat((idx, idx_next), dim=1)

    return idx

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # unsqueeze(0)はバッチ次元を追加する
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # バッチ次元を削除
    return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)
print(f"Output text:\n {token_ids_to_text(token_ids, tokenizer)}")

In [ ]:
# 損失関数の実装